# Compiling the RISC-V GCC Toolchain on your PYNQ Board

This notebook describes the steps to compile the RISC-V GCC Toolchain on your PYNQ Board. This depends on the steps in **[Downloading Dependencies](1-Downloading-And-Configuring.ipynb)**. You must complete the steps in that notebook before attempting this notebook.

This notebook is run entirely on a PYNQ board. The cells within this notebook will take several hours to finish - so be patient. On the bright side, this notebook can (and should) be run in parallel with the host-based **[Creating a FPGA Bitstream](2-Creating-A-Bitstream.ipynb)**.

These cells are based on the instructions provided in the [riscv-gnu-toolchain repository](https://github.com/riscv/riscv-gnu-toolchain)

# Verifying the riscv-gnu-toolchain Repository

You must have the riscv-gnu-toolchain repository cloned in order for this notebook to run. The following cell checks for the existence of the riscv-gnu-toolchain git repository in the `/home/xilinx` folder on the PYNQ board.

If these commands fail return to **[Downloading Dependencies and Configuring your Environment](1-Downloading-And-Configuring.ipynb)** and re-run the steps there.

In [1]:
!ls -lh /home/xilinx/riscv-gnu-toolchain

total 328K
drwxr-xr-x 16 root root 4.0K Nov 16 21:21 build-binutils-newlib
drwxr-xr-x 11 root root 4.0K Nov 16 22:39 build-gcc-newlib-stage1
drwxr-xr-x 13 root root 4.0K Nov 17 05:26 build-gcc-newlib-stage2
drwxr-xr-x  4 root root 4.0K Nov 17 01:51 build-newlib
-rw-r--r--  1 root root 9.3K Nov 16 20:51 config.log
-rwxr-xr-x  1 root root  25K Nov 16 20:51 config.status
-rwxr-xr-x  1 root root 129K Nov 16 18:46 configure
-rw-r--r--  1 root root 5.2K Nov 16 18:46 configure.ac
-rw-r--r--  1 root root  41K Nov 16 18:46 LICENSE
drwxr-xr-x  3 root root 4.0K Nov 16 18:46 linux-headers
-rw-r--r--  1 root root  20K Nov 16 20:51 Makefile
-rw-r--r--  1 root root  20K Nov 16 18:46 Makefile.in
-rw-r--r--  1 root root 4.1K Nov 16 18:46 README.md
drwxr-xr-x  2 root root 4.0K Nov 16 18:46 regression
drwxr-xr-x 22 root root 4.0K Nov 16 19:00 riscv-binutils-gdb
drwxr-xr-x  8 root root 4.0K Nov 16 19:00 riscv-dejagnu
drwxr-xr-x 37 root root 4.0K Nov 16 19:58 riscv-gcc
drwxr-xr-x 69 root root 4.0K Nov 16 2

# Building the RISC-V Tools

Now that we've met all our dependencies, we need to compile and install the tools. In the next two cells we build, and then install the tools. 

Once the tools have been compiled you will need to modify the Linux environment PATH variable and then reboot your board in order for the change to take effect. Once you can, we verify the tools and complete the notebook.

If you want to monitor the progress of your compilation you can read the config.log and build.log file that is generated during. You can read the logs by opening [cnfig.log](cnfig.log) and [build.log](build.log) by clicking the links or by opening them in the [Jupyter Browser](.). Update the logs by running refresh periodically.

Alternatively, you can also view the logs by opening a terminal by clicking [here](../../../terminals/42) or by navigating to (**Jupyter Home** > **New** > **Terminal**) and then running the commands: 
    
    less /home/xilinx/jupyter_notebooks/RISC-V-Tutorial/config.log

or

    less /home/xilinx/jupyter_notebooks/RISC-V-Tutorial/build.log

In [2]:
from subprocess import run, PIPE, Popen

cnfig_log = open("cnfig.log","w")
cnfig_err = open("cnfig.err","w")

ret = run(["./configure", "--prefix=/opt/riscv32im", "--with-arch=rv32im"], 
          cwd="/home/xilinx/riscv-gnu-toolchain/",
          stdout=cnfig_log, stderr=cnfig_err)
cnfig_log.close()
cnfig_err.close()

if(ret.returncode != 0):
    print("There was an error while configuring the RISCV Toolchain. See the config.err and config.log file")
else:
    print("Configuration successful!")

Configuration successful!


In [2]:
build_log = open("build.log","w")
build_err = open("build.err","w")    
ret = run(["make"], 
          cwd="/home/xilinx/riscv-gnu-toolchain/",
          stdout=build_log, stderr=build_err)
build_log.close()
build_err.close()

if(ret.returncode != 0):
    print("There was an error while compiling the RISCV Toolchain. See the build.err and build.log file")
else:
    print("Compile successful!")

Compile successful!


# Modifying PATH

Now that the tools have been installed we need to add them to the PATH variable so that they can be run from Jupyter Notebooks and the command line. To do this we will modify the `/etc/environment` file using the cell below.

If the PATH already contains the folder with the RISC-V GCC binaries this command will succeed but make no changes.


In [3]:
import os
path = os.environ['PATH'].split()
riscv_path = '/opt/riscv32im/bin'
if(riscv_path not in path):
    print('Updating /etc/environment file... ',end="")
    !sed -i 's/PATH=\"\(.*\)\"/PATH=\"\/opt\/riscv32im\/bin:\1\"/' /etc/environment
    print('done')
else:
    print("/etc/environment file already updated")

Updating /etc/environment file... done


# Reboot...

Reboot your board once the cell above has run. When your board finishes rebooting refresh this page and proceed to the next cell.

In [4]:
!shutdown -r now

# Verify Installation


The following cell verifies that the RISC-V Toolchain has been successfully installed. If you see an error message, make sure that your toolchain compiliation completed succesffully.


In [1]:
!riscv32-unknown-elf-gcc --version

riscv32-unknown-elf-gcc (GCC) 7.1.1 20170509
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

